# OSCARS Tutorial for the SR Module

This is a brief getting started tutorial for the oscars.sr module.  The oscars.sr module is for computing quantaties related to the radiative properties of charged particles in motion.  It is concerned with *REAL* field data for magnetic and electric fields.  At its root it is a discritization of the field equations which are derived from the Liénard–Wiechert potentials.  It is valid in the near-field and for relativistic and non-relativistic particles alike.

In addition to the oscars.sr module there exists an oscars.th module which deals more with theoretical analytical results for bending magnets, undulators, wigglers, spectra, flux, and brightness.

In this short tutorial only the basics on getting started with OSCARS are explored.  You can find more information on the main oscars website at:

https://oscars.bnl.gov

many examples at:

https://oscars.bnl.gov/examples.php

and full documentation at:

https://oscars.bnl.gov/doc.php

The documentation is also available within this notebook by adding a ? to the end of the function name and executing that cell, *e.g.*:

osr.calculate_spectrum?

## Step 0 - Importing oscars.sr

The zero-th step is to import oscars.sr and create an oscars.sr object.  For this tutorial we will set nthreads=8 in creating the object which will signify that we want to use 8 threads for all calculations.  We will also import some oscars plotting utilities and ask that the plots be displayed inline in the notebook.

In [ ]:
# Since we'll be using matplotlib we will ask that plots be 'inline' in the notebook
%matplotlib inline

# Import the oscars.sr module
import oscars.sr

# Import oscars plotting utilities (not needed, but convenient)
from oscars.plots_mpl import *
from oscars.plots3d_mpl import *

# Create the oscars.sr object with nthreads=8
osr = oscars.sr.sr(nthreads=8)

## Step 1 - Creating an NSLSII beam

We now create a beam silimar to NSLSII.  For the moment this will be a filament beam, but OSCARS does understand the Twiss parameters for non-zero emittance beams.

All directions and energies are configurable, but for now we'll use the defaults (beam pointing in the Z-direction).

You MUST specify the initial position and start and stop times for the calculation (the start time can be before the initial time t0 at the initial position x0.  The default for t0 is 0 for any x0).

Many things in OSCARS are 3D points or vectors, for instance the position x0=[0, 0, -1].

In [ ]:
# Setup the standard NSLSII beam conditions starting at z=-1 with the beam in the Z-direction
osr.set_particle_beam(beam='NSLSII', x0=[0, 0, -1])
osr.set_ctstartstop(0, 2)

# Print the oscars object properties thus far
osr.print_all()

## Step 2 - Add a simple undulator (or other structure)

There are many builtin bfield (and efield) types.  Here we will use a basic sinusoidal undulator.  One may note that the period is a vector and not a scalar.  This allows one to have the direction of the magnetic structure any way they like (typically it is along the beam direction).

Other basic fields are:
* osr.add_bfield_uniform()
* osr.add_bfield_gaussian()
* osr.add_bfield_function()
* osr.add_bfield_file()
* osr.add_bfield_interpolated()
* osr.add_bfield_quadrupole()

In [ ]:
# Typically a good idea to clear all bfields before adding
osr.clear_bfields()

# Add a basic undulator with peak field By of 1 [T], period 20 [mm], and 31 periods
osr.add_bfield_undulator(bfield=[0, 1, 0], period=[0, 0, 0.040], nperiods=31)

# Plot the field components
plot_bfield(osr)

## Step 3 - Calculate Trajectory

Calculate the trajectory based on the beam and fields added.  This step is not necessary, but very useful for debugging.

In [ ]:
# Calculate Trajectory
trajectory = osr.calculate_trajectory()

# Plot trajectory position and velocity
plot_trajectory_position(trajectory)
plot_trajectory_velocity(trajectory)

## Step 4 - Quantities of Interest

The stage is now set and you may run a number of calculations of interest.

### Spectra

Calculate a basic spectrum at an observation point 30 [m] downstream.

In [ ]:
# Calculate spectrum
spectrum = osr.calculate_spectrum(obs=[0, 0, 30], energy_range_eV=[10, 2000])

# Plot basic spectrum
plot_spectrum(spectrum)

In [ ]:
# Zoom in on 3rd harmonic
spectrum_3h = osr.calculate_spectrum(obs=[0, 0, 30], energy_range_eV=[780, 830])

# Plot basic spectrum
plot_spectrum(spectrum_3h)

### Calculate Flux Map for 3rd Harmonic

Here we cancluate a simple flux map.  The viewing plane is created in the X-Y plane and translated downstream 30 [m].  All objects in oscars can be 'translated' (viewing planes, magnetic fields, electric fields, beams, etc) as well as rotated with the 'rotations' argument.

In [ ]:
# Calculate flux map 30 [m] downstream
flux_3h = osr.calculate_flux_rectangle(plane='XY',
                                       energy_eV=803,
                                       width=[0.01, 0.01],
                                       npoints=[101, 101],
                                       translation=[0, 0, 30]
                                      )

# Plot the flux map
plot_flux(flux_3h)

In [ ]:
# Alternatively plot both with a line
plot_flux_spectrum(flux_3h, spectrum_3h, energy=803)

### Calculate Power Density

Similarly we will calculate the power density on a viewing plane translated 30 [m] downstream where the photon beam is relatively perpendicular to the viewing plane.  In the second power density a plane tangential to the beam is created and translated downstream (simulating say a beampipe upper wall)

Note in the second power density the axes are shifted.  You can retrive the 3D coordinates using the argument 'dim=3' (dim=2 is the default as it is useful for plotting)

In [ ]:
# Calculate power density on perpendicular plane
pd_1 = osr.calculate_power_density_rectangle(plane='XY',
                                             width=[0.05, 0.05],
                                             npoints=[101, 101],
                                             translation=[0, 0, 30]
                                            )

# Plot power density
plot_power_density(pd_1, xlabel='X [m]', ylabel='Y [m]')

In [ ]:
# Calculate power density on parallel plane, starting at the center of the ID
# spanning 2 [m] downstream, slightly offset vertically
pd_2 = osr.calculate_power_density_rectangle(plane='XZ',
                                             width=[0.02, 2.00],
                                             npoints=[51, 101],
                                             translation=[0, 0.004, 1]
                                            )

# Plot power density
plot_power_density(pd_2)

### Power Density on CAD Surface (STL format)

Now we will calculate the power density on a simple CAD object in STL format.

In [ ]:
# Calculate the power density on CAD STL sphere
pd_sphere = osr.calculate_power_density_stl(ifile='data/sphere.stl',
                                            scale=0.001,
                                            translation=[0, 0, 10]
                                           )

In [ ]:
# Plot power density 3D for STL
plot_power_density_stl(pd_sphere)

## Multi-particle Simulations

If you specify more realistic beam parameters you can run meaningful multi-particle simulations.  The example below shows the same spectrum and flux map from above with a few more particles averaged.  For this we will create a new oscars.sr object so as not to confuse it with the previous one.

Note: All that is needed is to add the argument 'nparticles=N' where N is the number of particles you want in the smulation.

In [ ]:
# Create new oscars.sr object for multi-particle simulations
osr_ss = oscars.sr.sr(nthreads=8)

# Set beam parameters for short straight at NSLSII
osr_ss.set_particle_beam(beam='NSLSII-ShortStraight', x0=[0, 0, -1])
osr_ss.set_ctstartstop(0, 2)

# Set undulator same as previous
osr_ss.add_bfield_undulator(bfield=[0, 1, 0], period=[0, 0, 0.040], nperiods=31)

In [ ]:
# Calculate sample spectrum for multi-particle simulation
spectrum_mp_3h = osr_ss.calculate_spectrum(obs=[0, 0, 30],
                                           energy_range_eV=[780, 830],
                                           nparticles=10
                                          )

# Plot spectra comparison
plot_spectra([spectrum_3h, spectrum_mp_3h], ['single-electron', 'multi-electron'])

In [ ]:
# Run multi-particle flux calculation
flux_mp = osr_ss.calculate_flux_rectangle(plane='XY',
                                          energy_eV=803,
                                          width=[0.01, 0.01],
                                          npoints=[101, 101],
                                          translation=[0, 0, 30],
                                          nparticles=3
                                         )

# Plot the flux map
plot_flux(flux_mp)

## Imported Real-Field Data

It is easy to import (and export) real field data in several formats.  It is also possible to read a list of files and interpolate between some parameter (say gap of phase).

Below is an example of importing a real data file from magnetic measurements of ESM EPU105, placing the device in the straight section where it belongs, and producing a power density map, and saving that map to a file.

In [ ]:
# Create new oscars.sr object for multi-particle simulations
osr_esm = oscars.sr.sr(nthreads=8)

# Set beam parameters for short straight at NSLSII
osr_esm.set_particle_beam(beam='NSLSII', x0=[0, 0, -1])
osr_esm.set_ctstartstop(0, 4)

# Set undulator same as previous
osr_esm.add_bfield_file(ifile='data/ESM_G16.0mPPh-30.00.txt',
                        iformat='OSCARS1D Z Bx By Bz',
                        scale=[0.001, 1, 1, 1],
                        translation=[0, 0, -1.205]
                       )

# Plot bfield
plot_bfield(osr_esm, -1, 3)

# Plot the trajectory as a check
plot_trajectory_position(osr_esm.calculate_trajectory())

In [ ]:
# Calculate power density
pd_esm = osr_esm.calculate_power_density_rectangle(plane='XY',
                                                   width=[0.1, 0.1],
                                                   npoints=[101, 101],
                                                   translation=[0, 0, 30],
                                                   ofile='power_density_esm.dat'
                                                  )

# Plot power density
plot_power_density(pd_esm, xlabel='X [m]', ylabel='Y [m]', ofile='power_density_esm.pdf')